#1. 데이터셋 얻기

> train에는 데이터가 label에는 결과 정보가 들어가 있다. 위와 같이 이미지와 label 정보를 확인할 수 있다.

#2. 모델 만들기

##2.1 모델 틀 생성

>모델을 생성하는데 Sequential()은 순차적으로 레이러를 추가할 수 있는 모델을 뜻한다.

##2.2 layer 추가

> input_shape는 입력 데이터를 뜻하는 것으로 (*, 28 * 28) 형식의 모든 데이터를 입력 받는 것이다.

> 512는 입력받은 데이터를 (*, 512) 형식으로 출력하는 결과이다.

> relu는 이미지 분석 시 기본적으로 사용되는 활성화 함수이다. 이 함수를 이용하면 0이하의 데이터는 0으로 나머지는 입력된 값으로 결과를 얻어 낼 수 있다. 또한 layer가 많은 경우 발생할 수 있는 gradient vanishing(기울기 사라짐) 문제를 해결할 수 있다.

> 두 번째 layers는 기존에 사용했던 input_shape이 유지 되며 softmax는 one-hot encoding을 적용하여 다양한 결과를 도출해 낼 수 있다. 여기서는 0~9까지 총 10개의 수를 분류해야 함으로 10을 사용하였다.

> softmax는 일반 벡터를 확률 벡터로 변환하는 것으로 주사위 1/6, 1/6 형식으로 표현하는 것과 같다. 이렇게 표현되면 1~6까지 6개의 데이터가 표현되는 것을 구현할 수 있는 것이다.

>https://keras.io/ko/layers/core/

##2.3 컴파일

> 최적화는 rmsprop를 사용한다. 보편적으로 많이 사용되는 것은 atom이며 atom이전에 많이 사용되었던 최적화 도구이다. 두 부분을 변경하여 사용해 보자.

> 손실함수는 여러 결과를 얻어야 함으로 categorical_crossentropy를 사용했다. 

> 지표는 정확도를 사용했다.

#3. 데이터 변환

> 일반적으로 이미지 데이터는 가로, 세로, 체널(색상정보)로 이루어진 3D 데이터이지만 본 데이터는 흑백사진으로 채널이 1인 데이터라 이를 제거하고 2D 형식으로 shape 정보를 가지고 있다.

> 원본 데이터는 0부터 255사이의 데이터이다. 수치를 실수화 처리하여 정확하고 빠른 확률 정보를 얻을 수 있다.

#4. 모델 학습 및 평가


##4.1 모델 학습하기
> 에포크는 적당히 5로 지정했으며 batch_size도 적당히 128로 지정했다.

##4.2 평가하기

> 검증을 위해 test 데이터를 변환한다.

> 검증결과 손실함수는 0.06 정도로 나타나며 정확도는 0.98으로 나타난다.

#5. 예측하기

> ont-hot encoding으로 처리 했음으로 이중 가장 높은 활률의 데이터 위치가 실제 데이터가 된다. 즉, 7이 첫 번째 예측값의 결과인 것이다.

> 실제값과 예측값을 비교해 보면 같이 나타난 것을 볼 수 있다.

#6. 모델 저장하기